In [ ]:
import reactionmodel.parser
from reactionmodel.model import Model, eval_expression
import numpy as np

In [ ]:
from reactionmodel.model import Reaction, Species
class DummyModel(Model):
    def __init__(self, species: list[Species], reactions: list[Reaction]) -> None:
        super().__init__(species, reactions, reject_duplicates=False)

parsed = reactionmodel.parser.load('./abr.yaml', model_class=DummyModel)
m = parsed.model

In [ ]:
m.get_k(parameters=parsed.parameters)

In [ ]:
good = []
zero_propensity = []
noop = []
for r in m.all_reactions:
    if eval_expression(r.k, parsed.parameters) == 0:
        zero_propensity.append(r)
        continue
    elif r.reactants == r.products:
        noop.append(r)
    else:
        good.append(r)


In [ ]:
len(good), len(zero_propensity), len(noop)

## Reduced model

In [ ]:
reduced_model = Model(m.species, good)

In [ ]:
dydt = reduced_model.get_dydt(parameters=parsed.parameters, sparse=True)

In [ ]:
initial = reduced_model.make_initial_condition({'<M_M_M>': 1.0})

In [ ]:
initial.shape

In [ ]:
dydt(0, initial)

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
result = solve_ivp(dydt, [0, 100.0], initial)

In [ ]:
legend = []
for i, quantity in enumerate(result.y[:, -1]):
    if quantity > 5e-2:
        legend.append(reduced_model.species[i])
    else:
        legend.append('_nolegend_')

In [ ]:
plt.plot(result.t, result.y.T)
plt.legend(legend)

## Timing tests

In [ ]:
x = np.random.rand(64).reshape(64,1)

In [ ]:
%%timeit
m.stoichiometry() * x

In [ ]:
import scipy.sparse

csr = scipy.sparse.csr_array(m.stoichiometry())
bsr = scipy.sparse.bsr_array(m.stoichiometry())

In [ ]:
%%timeit
bsr * x

In [ ]:
%%timeit
csr * x

In [ ]:
len([r for r in m.all_reactions if not (r.reactants == r.products)])

In [ ]:
len(set(m.all_reactions))

In [ ]:
for r in m.all_reactions:
    if 'displaces' in r.description:
        print(r)
        break

In [ ]:
parsed.parameters

In [ ]:
m.get_k(parameters=parsed.parameters)